### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [4]:
schools = pd.unique(school_data_complete['school_name'])
num_of_schools = len(schools)
print(f'Number of Schools: {num_of_schools}')

students = pd.unique(school_data_complete['Student ID'])
num_of_students = len(students)
print(f'Number of Students: {num_of_students}')

total_budget = school_data['budget'].sum()
print(f'Total Budget: {total_budget}')

average_math_score = round(student_data['math_score'].mean(), 2)
print(f'Average Math Score: {average_math_score}')

average_reading_score = round(student_data['reading_score'].mean(), 2)
print(f'Average Reading Score: {average_reading_score}')

percent_passing_math = round( (len(student_data[student_data['math_score'] >= 70]) / num_of_students) * 100, 2) 
print(f'% Passing Math: {percent_passing_math}')

percent_passing_reading = round( (len(student_data[student_data['reading_score'] >= 70]) / num_of_students) * 100, 2) 
print(f'% Passing Reading: {percent_passing_reading}')

percent_passing_overall = len( student_data[ ( student_data['math_score'] >= 70 ) & ( student_data['reading_score'] >= 70 ) ] ) / num_of_students
percent_passing_overall = round(percent_passing_overall * 100, 2)
print(f'% Overall Passing: {round(percent_passing_overall * 100, 2)}')

Number of Schools: 15
Number of Students: 39170
Total Budget: 24649428
Average Math Score: 78.99
Average Reading Score: 81.88
% Passing Math: 74.98
% Passing Reading: 85.81
% Overall Passing: 6517.0


In [5]:
distrcit_summary_df = pd.DataFrame(
    {
        'Total Schools': [f'{num_of_schools:,}'],
        'Total Students': [f'{num_of_students:,}'],
        'Total Budget': [f'${total_budget:,.2f}'],
        'Average Math Score': [f'{average_math_score:,}'],
        'Average Reading Score': [f'{average_reading_score:,}'],
        '% Passing Math': [f'{percent_passing_math:,}'],
        '% Passing Reading': [f'{percent_passing_reading:,}'],
        '% Overall Passing': [f'{percent_passing_overall:,}']
    },
)

distrcit_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98,85.81,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [6]:
# Drop the id column from school data and rename the other columns
school_summary_df = school_data.drop('School ID', axis=1)
school_summary_df = school_summary_df.rename(columns = {
    'type': 'School Type', 
    'size':'Total Students', 
    'budget':'Total School Budget'
})

# Set the index
school_summary_df = school_summary_df.set_index('school_name')

# Calculate the per student budget column
school_summary_df['Per Student Budget'] = school_summary_df['Total School Budget'] / school_summary_df['Total Students']

# Calculate the average math score and average reading score columns
mean_scores_by_school = student_data.groupby('school_name')[['math_score', 'reading_score']].mean()
mean_scores_by_school = mean_scores_by_school.rename(columns={'math_score': 'Average Math Score', 'reading_score': 'Average Reading Score'})
school_summary_df = school_summary_df.join(mean_scores_by_school)

# Calculate percent passing math column
count_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name'])[['math_score']].count()
count_passing_math = count_passing_math.rename(columns={'math_score': '% Passing Math'})
school_summary_df = school_summary_df.join(count_passing_math)
school_summary_df['% Passing Math'] = (school_summary_df['% Passing Math'] / school_summary_df['Total Students']) * 100

# Calculate percent passing reading column
count_passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70].groupby(['school_name'])[['reading_score']].count()
count_passing_reading = count_passing_reading.rename(columns={'reading_score':'% Passing Reading'})
school_summary_df = school_summary_df.join(count_passing_reading)
school_summary_df['% Passing Reading'] = (school_summary_df['% Passing Reading'] / school_summary_df['Total Students']) * 100

# Calculate percent overall passing column
count_passing_overall = school_data_complete[(school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)].groupby(['school_name'])[['reading_score']].count()
count_passing_overall= count_passing_overall.rename(columns={'reading_score':'% Overall Passing'}) 
school_summary_df = school_summary_df.join(count_passing_overall)
school_summary_df['% Overall Passing'] = (school_summary_df['% Overall Passing'] / school_summary_df['Total Students']) * 100

format = {
    'Total Students': '{:,}',
    'Total School Budget': '${:,}',
    'Per Student Budget': '${:,.2f}',
    'Per Student Budget':'${:,.2f}',
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2f}',
    '% Passing Reading': '{:.2f}',
    '% Overall Passing': '{:.2f}'
}

school_summary_df = school_summary_df.sort_index()
school_summary_df.style.format(format)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,"4,976","$3,124,928",$628.00,77.05,81.03,66.68,81.93,54.64
Cabrera High School,Charter,"1,858","$1,081,356",$582.00,83.06,83.98,94.13,97.04,91.33
Figueroa High School,District,"2,949","$1,884,411",$639.00,76.71,81.16,65.99,80.74,53.20
Ford High School,District,"2,739","$1,763,916",$644.00,77.10,80.75,68.31,79.30,54.29
Griffin High School,Charter,"1,468","$917,500",$625.00,83.35,83.82,93.39,97.14,90.60
Hernandez High School,District,"4,635","$3,022,020",$652.00,77.29,80.93,66.75,80.86,53.53
Holden High School,Charter,427,"$248,087",$581.00,83.80,83.81,92.51,96.25,89.23
Huang High School,District,"2,917","$1,910,635",$655.00,76.63,81.18,65.68,81.32,53.51
Johnson High School,District,"4,761","$3,094,650",$650.00,77.07,80.97,66.06,81.22,53.54


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [13]:
school_summary_df.sort_values(by='% Overall Passing', ascending=False).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [14]:
school_summary_df.sort_values(by='% Overall Passing', ascending=True).head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [36]:
grade_9_df = student_data[student_data['grade'] == '9th'].groupby('school_name')[['math_score']].mean()
grade_10_df = student_data[student_data['grade'] == '10th'].groupby('school_name')[['math_score']].mean()
grade_11_df = student_data[student_data['grade'] == '11th'].groupby('school_name')[['math_score']].mean()
grade_12_df = student_data[student_data['grade'] == '12th'].groupby('school_name')[['math_score']].mean()

reading_scores_by_grade = pd.DataFrame({
    '9th': grade_9_df['math_score'],
    '10th': grade_10_df['math_score'],
    '11h': grade_11_df['math_score'],
    '12th': grade_12_df['math_score']
})
reading_scores_by_grade


,9th,10th,11h,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
grade_9_df = student_data[student_data['grade'] == '9th'].groupby('school_name')[['reading_score']].mean()
grade_10_df = student_data[student_data['grade'] == '10th'].groupby('school_name')[['reading_score']].mean()
grade_11_df = student_data[student_data['grade'] == '11th'].groupby('school_name')[['reading_score']].mean()
grade_12_df = student_data[student_data['grade'] == '12th'].groupby('school_name')[['reading_score']].mean()

reading_scores_by_grade = pd.DataFrame({
    '9th': grade_9_df['reading_score'],
    '10th': grade_10_df['reading_score'],
    '11h': grade_11_df['reading_score'],
    '12th': grade_12_df['reading_score']
})
reading_scores_by_grade

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [63]:
# create bins and labels for the budget ranges
bins = [0,585, 630, 645, 680]
group_labels = ["Less Than $585", "$586-630", "$631-645", "$646 to 680"]

#slcie data and place into bins
budget_bins = pd.cut(school_summary_df["Per Student Budget"], bins, labels=group_labels)

# add the budget ranges to the school data
school_summary_df['Budget Range'] = budget_bins

# group data by budget range
budget_groups = school_summary_df.groupby(['Budget Range'])

# create a df for the budget ranges
budget_df = budget_groups[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']].mean()

format = {
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2f}',
    '% Passing Reading': '{:.2f}',
    '% Overall Passing': '{:.2f}'
}

budget_df.style.format(format)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Budget Range,,,,,
Less Than $585,83.46,83.93,93.46,96.61,90.37
$586-630,81.90,83.16,87.13,92.72,81.42
$631-645,78.52,81.62,73.48,84.39,62.86
$646 to 680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [62]:
# create bins and labels for school size ranges
bins = [0, 1000, 2000, 5000]
group_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#slcie data and place into bins
size_bins = pd.cut(school_summary_df["Total Students"], bins, labels=group_labels)

# add the budget ranges to the school data
school_summary_df['School Size Range'] = size_bins

# group data by budget range
size_groups = school_summary_df.groupby(['School Size Range'])

# create a df for the budget ranges
size_df = size_groups[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']].mean()

format = {
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2f}',
    '% Passing Reading': '{:.2f}',
    '% Overall Passing': '{:.2f}'
}

size_df.style.format(format)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size Range,,,,,
Small (<1000),83.82,83.93,93.55,96.10,89.88
Medium (1000-2000),83.37,83.86,93.60,96.79,90.62
Large (2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

* Perform the same operations as above, based on school type

In [61]:
type_df = school_summary_df.groupby(['School Type']).mean()[['Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading', '% Overall Passing']]
format = {
    'Average Math Score': '{:.2f}',
    'Average Reading Score': '{:.2f}',
    '% Passing Math': '{:.2f}',
    '% Passing Reading': '{:.2f}',
    '% Overall Passing': '{:.2f}'
}

type_df.style.format(format)


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
